<div style="font-size: 200%; font-weight: bold; color: maroon;">Procesado de la red de vuelos con GraphFrames (II)</div>
<!-- Paulo Villegas, 2020 - v. 1.2 -->

Uso de los **[algoritmos predefinidos](https://graphframes.github.io/graphframes/docs/_site/user-guide.html#graph-algorithms)** en Graphframes

In [ ]:
spark.conf.get('spark.master')

# Leer datos

Regeneramos el grafo desde los DataFrames grabados a disco

In [ ]:
import graphframes as gf

vertices = spark.read.parquet('OUT/flights-vertices')
edges = spark.read.parquet('OUT/flights-edges')

g = gf.GraphFrame(vertices, edges)

In [ ]:
g.edges.count()

In [ ]:
g.edges.printSchema()

# Subgrafos

Calculamos un subgrafo: la parte del grafo que corresponde a los vuelos de Iberia

In [ ]:
e2 = g.edges.filter('Airline = "IB"')

In [ ]:
g2 = gf.GraphFrame(g.vertices, e2)

In [ ]:
g2.edges.count()

# Label propagation

Calculamos comunidades en el grafo mediante el algoritmo LPA

In [ ]:
%%time 
result = g.labelPropagation(maxIter=5)

In [ ]:
result.select("id", "label").show()

In [ ]:
result.printSchema()

Veamos el tamaño de las comunidades que han resultado

In [ ]:
result.groupBy('label').count().sort('count', ascending=False).limit(60).toPandas()

Pasamos alguna de las comunidades creadas a local, para poder analizarlas más cómodamente

In [ ]:
result.filter('label = 3615').toPandas()

Vamos a representar comunidades colocando los nodos en un mapa

In [ ]:
# Nos traemos la definición de las comunidades
clusters = result.toPandas()

In [ ]:
# Elegimos la comunidad a representar
LABEL = 

import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

%matplotlib inline
plt.style.use('bmh')
plt.rcParams['figure.figsize'] = (16, 10)

data_crs = ccrs.PlateCarree()
crs = ccrs.Mollweide()

fig, ax = plt.subplots(1, 1, figsize=(20, 14), subplot_kw=dict(projection=data_crs))
ax.coastlines()
ax.stock_img()

group = clusters[clusters.label == LABEL]
ax.scatter([a.Longitude for a in group.itertuples()], [a.Latitude for a in group.itertuples()],
           color='red', transform=data_crs);

# PageRank

Cálculo del valor de PageRank para la red de aeropuertos

In [ ]:
%%time
pr = g.pageRank(resetProbability=0.1, tol=0.01)

In [ ]:
type(pr)

Como vemos, en este caso el gresultado no es un DataFrame, sino un GraphFrame completo:
* a cada nodo se le añade una columna con su valor de PageRank
* a cada enlace se añade su peso calculado

In [ ]:
pr.vertices.printSchema()

In [ ]:
pr.edges.printSchema()

In [ ]:
pr.vertices.select("id", "pagerank").show()
pr.edges.select("src", "dst", "weight").show()

Ordenamos los nodos por su valor de PageRank

In [ ]:
pr_order = pr.vertices.select('id', 'Name', 'City', 'pagerank').orderBy('pagerank', ascending=False)

In [ ]:
pr_order.limit(20).toPandas()

In [ ]:
pr_order.printSchema()

## PageRank y grados
Ahora comparamos el orden inducido por PageRank con el del grado del nodo

In [ ]:
pr_order_deg = pr_order.join(g.degrees, 'id')

In [ ]:
pr_order_deg.sort('pagerank', ascending=False).limit(20).toPandas()